In [1]:
import numpy as np
import pandas as pd

Players = ["P0","P1"]
MatchType = "Challenging"

decks_df = pd.read_csv("https://raw.githubusercontent.com/amcclead7336/MTG_GameData/main/Decks.csv")

i=0
while True:
    rand_selects = decks_df.sample(len(Players))
    diff = abs(rand_selects.iloc[0][3] - rand_selects.iloc[1][3])
    
    if MatchType=="Equal" and diff == 0:
        break
    if MatchType=="Fair" and diff <= 1:
        break
    if MatchType=="Challenging" and diff == 2:
        break
    if MatchType=="Crazy" and diff > 2:
        break
    i+=1
    if i>1000:
        print("Couldn't find match")
        break
        
for i in range(len(Players)):
    print(Players[i]+"'s Deck is: ")
    print("\t* Deck: {0}\n\t* ID: {1}\n\t* Rating: {2}\n".format(rand_selects.iloc[i][1],rand_selects.iloc[i][0],rand_selects.iloc[i][3]))

P0's Deck is: 
	* Deck: Riveteer Rampage
	* ID: 9
	* Rating: 4

P1's Deck is: 
	* Deck: Dungeons of Death
	* ID: 4
	* Rating: 2

